# Monte Carlo Method 

In [63]:
import pandas as pd
import re # regex
import time
from random import random
from itertools import chain, combinations

## Building the input DataFrames
* a DataFrame *general_df* for general project data (link to base data and inital value generator)
* a DataFrame *project_df* with project data, like number of jobs and the due date
* a DataFrame *resources_df* which is a list of all resources and their limits
* a DataFrame *df* that contains all the network info: all nodes with their successors and predecessors, their duration, etc.
* a DataFrame *limits* that is used to keep track of available resources and resources in use

In [64]:
#path = r'D:\Gregor\Dropbox\Studium\j120.sm\j12060_9.sm'
#path = r'C:\Users\webgr_000\Dropbox\Studium\[M04] - WS 2018-2019\Thesis\Test Sets\J30\J30.sm\j301_1.sm'
path = r'D:\Gregor\Dropbox\Studium\[M04] - WS 2018-2019\Thesis\Test Sets\J30\J30.sm\j301_1.sm'

with open(path, "r") as f:
    file = f.readlines()

In [65]:
general = {}
for line in file[1:3]:
    l = list(map(str.strip, line.split(sep = ':')))
    general[l[0]] = l[1]

project = {}
for line in file[4:7]:
    l = list(map(str.strip, line.split(sep = ':')))
    project[l[0]] = l[1]

info = (list(map(str.split, file[13:15])))
for i in range(len(info[0])):
    project[info[0][i]] = info[1][i]

general_df = pd.DataFrame.from_dict(general, orient='index', columns=['value'])
project_df = pd.DataFrame.from_dict(project, orient='index', columns=['value'])

In [66]:
res_sep = re.compile('[RND][ \d]{2}|[\d]+') # regex pattern to extract the resources
resources = {}
for line in file[8:11]:
    l = list(map(str.strip, line.replace('-', '').split(sep = ':')))
    resources[l[0]] = l[1][0]
res = [re.findall(res_sep, file[-3]), re.findall(res_sep, file[-2])]
for i, r in enumerate(res[0]):
    resources[r] = res[1][i]
resources_df = pd.DataFrame.from_dict(resources, orient='index', columns=['value'])

In [67]:
sep = re.compile('[*]+') # regex pattern to find the end of a block; blocks are seperated by a row of ****
dur_sep = re.compile('[RND][ \d]{2}|[\w.]+')

relations = []
idx = 17 # start index to find the precedence relations
while not re.findall(sep, file[idx]):
    relations.append(file[idx])
    idx += 1

durations = []
idx = idx + 4 # start index to find the task's durations. They start four lines after the relations block.
header = re.findall(dur_sep, file[idx - 2])
while not re.findall(sep, file[idx]):
    durations.append(file[idx].strip().split())
    idx += 1
durations_df = pd.DataFrame(durations, columns=header)

relations_df = pd.DataFrame(columns=relations[0].split())
for i, rel in enumerate(map(str.split, relations[1:])):
    relations_df.loc[i] = [rel[0], rel[1], rel[2], list(map(int, rel[3:]))]

# now merge/join both DataFrames and set the columns types to numeric (when possible)
df = relations_df.merge(durations_df, how='right', on='jobnr.')
df.set_index('jobnr.', inplace=True)

# find all the successors and add them as a new column to each state
succ = {'1':[]}
for index, row in df.iterrows():
    for successor in row['successors']:
        if str(successor) in succ:
            succ[str(successor)].append(int(index))
        else:
            succ[str(successor)] = [int(index)]
df['predecessors'] = df.index.map(succ)

df.index = df.index.map(int)
df = df.apply(pd.to_numeric, errors='ignore')

In [68]:
def get_limits(resources_df):
    limits = pd.concat([resources_df.iloc[3:], resources_df.iloc[3:]], axis = 1)
    #limits.columns = ['limits', 'available']
    return limits.apply(pd.to_numeric)

## Implementation of hyperparameter functions
* greediness *epsilon*
* rewards
* learning rate

Meyl uses different greediness parameters ε during theses phases. We define the function to determine the parameter:

$$\epsilon(n)=\left\{\begin{array}{ll} \varepsilon_s, & 0 \leq n \leq n_s \\
              \frac{\varepsilon_f - \varepsilon_s}{n_m}(n - n_s) + \varepsilon_s & n_s < n \leq n_s + n_m \text{, with } f_\varepsilon = 0 \\
              \varepsilon_m & n_s < n \leq n_s + n_m \text{, with } f_\varepsilon = 1 \\
              0 & n_s + n_m < n \leq n_{max} \end{array}\right.$$

In [69]:
def epsilon(n, function_type=0, eps_m=0):
    '''
    Returns the value of the greediness parameter epsilon that depends on the phase
    (exploration, learning, convergence) of the algorithm.

    Parameters
    ----------
    function_type: choose 0 for a constantly decaying and 1 for a constant value eps_m
    '''
    if 0 <= n <= n_s:
        return eps_s
    elif n_s < n <= n_m:
        if function_type:
            return eps_m
        else:
            return (((eps_f - eps_s) / n_m) * (n - n_s) + eps_s)
    elif n_m < n <= n_max:
        return 0

The reward function can use known project durations *t_min* and *t_max* to encourage the algorithm to improve good solutions even more.

$$r(t)=\left\{\begin{array}{ll} 1 - \left(\frac{t - t_{min}}{t_{max}}\right)^{0,4} & f_r = 0 \\
              \frac{t - t_{max}}{t_{min} - t_{max}} & f_r = 1 \end{array}\right.$$

In [70]:
def reward(t, function_type=0):
    '''
    Parameters
    ----------
    function_type: if the times t_min and t_max are unknown choose type 1 and calculate the
                    times manually by running a couple of random iterations
    '''
    if function_type:
        return (t - t_max) / (t_min - t_max)
    else:
        return 1 - ((t - t_min) / t_max)**0.4

Meyl implements two different types of learning rates: one uses a constant value around 0.01 and one uses a learning rate that decays linearly during the convergence phase (during (*n_s + n_m*) and *n_max*).

In [71]:
def learning_rate(n, function_type=0, alpha_f=0):
    '''
    Returns the learning rate alpha that is either a constant value alpha_b
    or can be chosen to decay during the convergence phase (for n > n_m + n_s).
    function_type -> choose 0 for a function that linearly decays from alpha_b
                     to a final value alpha_f during the convergence phase or
                     choose 1 for the constant value alpha_b.
    '''
    if 0 <= n <= n_m:
        return alpha_b
    elif n_m < n <= n_max:
        if function_type:
            return (((alhpha_b - alpha_f) / (n_m - n_max)) * (n - n_max)) + alpha_f
        else:
            return alpha_b

## Implementation of some helper functions

In [72]:
def get_actions_from_single_states(limits, finished_jobs, running_jobs, df):
    '''
    '''
    if epsilon(n, function_type=1, eps_m=0.2) < random():
        recorded_actions = actions.loc[(actions.possibleJobs ==
                    tuple(sorted(get_possible_jobs_from_single_states(finished_jobs, df)))) &
                    (actions.runningJobs == tuple(sorted(running_jobs.keys())))]
        if len(recorded_actions) > 0:
            return actions.loc[recorded_actions['q'].idxmax()].action

    feasible_actions = []
    # possible jobs are those that could be started because all their predecessors have been finished
    possible_actions = get_possible_jobs_from_single_states(finished_jobs, df)
    # we select only those actions that do not consume more resources than available from the possible_actions list 
    shuffled_actions = [key for key in possible_actions if min(limits.available.subtract(df.loc[key][limits.index])) >= 0]
    if len(shuffled_actions) > 0:
        shuffle(shuffled_actions)
        feasible_actions.append(shuffled_actions[0])

        for i in range(1, len(shuffled_actions)):
            # if we can add yet another activity from the shuffled_actions-list without using more resources than available
            # then add this activity to the feasible_actions-list.
            if min([x1 - x2 for (x1, x2) in zip(limits.available, [sum(row[column] for
                row in [df.loc[action][limits.index] for action in feasible_actions] +
                [list(df.loc[shuffled_actions[i]][limits.index])]) for column in range(len(limits.index))])]) >= 0:
                feasible_actions.append(shuffled_actions[i])

    return feasible_actions

def get_possible_jobs_from_single_states(finished_jobs, df):
    '''
    '''
    return set(chain(*[get_possible_jobs(job, finished_jobs, df) for job
                       in finished_jobs])) - set(running_jobs.keys()) - set(finished_jobs)

def get_possible_jobs(key, finished_jobs, df):
    '''
    A job can only be started when all of its predecessors are finished. This method returns - based on the job with id
    key and a list of already finished jobs - the jobs that could possibly be started.

    Parameters
    ----------
    key: a job id
    finished_jobs: a list of finished jobs
    df: the DataFrame with info about all jobs to get a job's successors and predecessors
    '''
    return [i for i in df.loc[key].successors if all(j in finished_jobs for j in df.loc[i].predecessors)]

def print_summary(t, n):
    '''
    Prints a small summary about the current status.

    Parameters
    ----------
    t: current time step
    n: the summary is printed each n time steps
    '''
    if t%n == 0 and t>0:
        print('iteration', t)
        print('running:', running_jobs)
        print('finished:', finished_jobs)

# Let's run a whole iteration
I will define all variables, initialise the problem and use the functions that were defined above

In [73]:
eps_s, eps_m, eps_f = 1, 0.2, 0.05
alpha_b = 0.1 # function_type = 0 -> constant value
n, n_max = 0, 1000
n_s, n_m = 0.03*n_max, 0.85*n_max
t_min, t_max = 44, 68
actions = pd.DataFrame(columns = ['possibleJobs', 'runningJobs', 'action', 'q'])
limits = get_limits(resources_df)
time_steps = []

* DataFrame *actions* keeps track of all the actions and is references by the q-table
* Dictionary *states* contains all states that are finished and the jobs that can be started in that state
* Dictionary *running_jobs* is used as a counter to calculate the remaining time until a job is finished
* List *finished_jobs* contains all finished jobs - a job can be started when all of its predecessors are in the finished_jobs list

In [74]:
start = time.time()

while n <= n_max:
    # initialisation
    t = 0
    running_jobs = {1:0}
    finished_jobs = []
    current_actions = pd.DataFrame(columns = ['possibleJobs', 'runningJobs', 'action'])

    # run while not all jobs are finished, i.e. while the length of the finished_jobs-list is less than the number of task
    while len(finished_jobs) < int(project_df.loc['jobs (incl. supersource/sink )']):
        acts = get_actions_from_single_states(limits, finished_jobs, running_jobs, df)
        if len(acts) > 0:
            current_actions = current_actions.append({
                'possibleJobs': tuple(sorted(get_possible_jobs_from_single_states(finished_jobs, df))),
                'runningJobs': tuple(sorted(running_jobs.copy().keys())),
                'action': tuple(sorted(acts))
            }, ignore_index = True)

            running_jobs.update({job: df.loc[job].duration for job in acts})
            limits.available = limits.available.subtract(
                pd.DataFrame(df.loc[key][limits.index] for key in acts).sum(axis='rows'), fill_value=0)

        ########################################################
        t += 1 # update timer and move on the the next time-step
        ########################################################

        for key in running_jobs.copy().keys():
            running_jobs[key] -= 1

            # free ressources if the task is finished and remove the task from the list of running jobs
            if running_jobs[key] <= 0:
                limits.available = limits.available.add(df.loc[key][limits.index], fill_value=0)
                running_jobs.pop(key)
                finished_jobs.append(key)

    for idx in range(len(current_actions)):
        row = current_actions.loc[idx]
        actions_row = actions.loc[(actions.possibleJobs == row.possibleJobs) & (actions.runningJobs == row.runningJobs) &
            (actions.action == row.action)]
        if len(actions_row) == 0:
            row['q'] = 0.0
            actions = actions.append(row, ignore_index=True)

        actions.loc[actions_row.index, 'q'] += learning_rate(n)**(len(current_actions) - idx) * (reward(t, function_type=1) - actions.loc[actions_row.index, 'q'])

    print_summary(t, 100)
    time_steps.append(t)
    n += 1
print(time.time() - start)

AttributeError: 'DataFrame' object has no attribute 'available'